# Deeplearning with cuda

## Imports

In [4]:
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, confusion_matrix

## Info about the codeblock bellow

Pytorch is able to run on GPU, but this is not enabled by default. We check if the GPU is available and enable it if it is.

In [5]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cpu



## Data preparation

In [6]:
# Set a random seed
random_seed = 42

# Import the Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)
print(X.shape, y.shape)

(150, 4) (150,)


Explenation of the codeblock bellow
torch.FloatTensor converts input to a float tensor (a tensor can only contain one type of data [documentation](https://pytorch.org/docs/stable/tensors.html) ) \
Float: 32-bit floating point \
Long: 64-bit integer (signed)

In [7]:
# Convert the dataset to PyTorch tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

## Creating the model

PyTorch models are created by subclassing `torch.nn.Module`. The model is defined by the `__init__` method, where the layers are defined, and the `forward` method, where the forward pass is defined. \
In the `__init__` method we define the layers of the model. \
__fc__ stands for fully connected layer. \
The layers available in PyTorch are defined in the `torch.nn` module. [Documentation](https://pytorch.org/docs/stable/nn.html) \
Since we want a fully connected layer, we use the [torch.nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) class. \
The forward method defines the forward pass of the model. \
In the forward method we define the operations that are performed on the input data. \
We use the [torch.nn.functional](https://pytorch.org/docs/stable/nn.functional.html) module to define the operations. \
The activation functions are defined in the `torch.nn.functional` module. \
We use the [torch.nn.functional.relu](https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.relu) function to apply the ReLU activation function. \


In [14]:
# Create a neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() # Call the parent class's constructor
        self.fc1 = nn.Linear(4, 100) # 4 input features, 100 output features
        self.fc2 = nn.Linear(100, 100) # 100 input features, 100 output features
        self.fc3 = nn.Linear(100, 3) # 100 input features, 3 output features

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5) # Added to show different loss in tensorboard
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [6]:


# Train the neural network

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

for epoch in range(100):
    optimizer.zero_grad()
    output = net(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

# Test the neural network
y_pred = net(X_test)
_, predicted = torch.max(y_pred, 1)

print(accuracy_score(y_test, predicted))
print(confusion_matrix(y_test, predicted))

# # Save the model
# torch.save(net.state_dict(), 'model.pth')

# # Load the model
# net = Net()
# net.load_state_dict(torch.load('model.pth'))
# net.eval()

1.0
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]


## Creating a writer for tensorboard

In [16]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir='runs/iris_experiment_1', comment='Iris dataset training')

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)


def train_model(iter):
    for epoch in range(iter):
        optimizer.zero_grad()
        y1 = model(X_train)
        loss = criterion(y1, y_train)
        writer.add_scalar("Loss/train", loss, epoch)
        writer.add_histogram("fc1.weight", model.fc1.weight, epoch)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print('Epoch:', epoch, 'Loss:', loss.item())

train_model(100)
writer.flush()
writer.close()

Epoch: 0 Loss: 1.11858069896698
Epoch: 10 Loss: 1.124367356300354
Epoch: 20 Loss: 1.1071430444717407
Epoch: 30 Loss: 1.1568927764892578
Epoch: 40 Loss: 1.1178781986236572
Epoch: 50 Loss: 1.077290415763855
Epoch: 60 Loss: 1.0895156860351562
Epoch: 70 Loss: 1.1391804218292236
Epoch: 80 Loss: 1.0739704370498657
Epoch: 90 Loss: 1.1220403909683228


Tensorboard is a tool that allows you to visualize the training process of your model. \
we can run it via the command `tensorboard --logdir=runs` where runs is the folder where runs are stored \
installing this is done via `pip install tensorboard` \
It might be worth looking into the tensorboard integration in pytorch with vs code. [here](https://code.visualstudio.com/docs/python/tensorboard) \
open this via command palette (ctrl+shift+p) and search for Python: Launch Tensorboard \

## herschrijving van de code die u had gegeven

Note 1: Zet je kaggle api in /root/.config/kaggle.json \

In [41]:
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd
import os

# load the minst fashion dataset from kaggle
if not os.path.exists('fashionmnist/fashion-mnist_test.csv'):
    !rm -rf fashionmnist
    !kaggle datasets download -d zalando-research/fashionmnist
    !mkdir fashionmnist
    !unzip fashionmnist.zip -d fashionmnist 

# Load the dataset
train_data = pd.read_csv('fashionmnist/fashion-mnist_train.csv')
test_data = pd.read_csv('fashionmnist/fashion-mnist_test.csv')

train_images = train_data.iloc[:, 1:].values
train_labels = train_data.iloc[:, 0].values

test_images = test_data.iloc[:, 1:].values
test_labels = test_data.iloc[:, 0].values

# Convert the dataset to PyTorch tensors
# numpy.ndarray to torch.tensor that contains float values

train_images = torch.FloatTensor(train_images)
train_labels = torch.LongTensor(train_labels)
test_images = torch.FloatTensor(test_images)
test_labels = torch.LongTensor(test_labels)


# Create a neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() # Call the parent class's constructor
        self.fc1 = nn.Linear(28*28, 128) # 784 input features, 128 output features
        self.fc2 = nn.Linear(128, 128) # 128 input features, 128 output features
        self.fc3 = nn.Linear(128, 10) # 128 input features, 10 output features

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.2)
        x = F.softmax(self.fc2(x))
        x = self.fc3(x)
        return x

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.01)

net = Net()

for epoch in range(10):
    net.optimizer.zero_grad()
    output = net(train_images)
    net.loss = criterion(output, train_labels)
    net.loss.backward()
    net.optimizer.step()
    if epoch % 10 == 0:
        print('Epoch:', epoch, 'Loss:', net.loss.item())
    
y_pred = net(test_images)
_, predicted = torch.max(y_pred, 1)

print(accuracy_score(test_labels, predicted))
print(confusion_matrix(test_labels, predicted))

AttributeError: module 'torch.nn' has no attribute 'categoryCrossEntropyLoss'

### Code met tensorboard

In [59]:
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd
import os
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir='runs/hassan', comment='Code provided by Hassan H.')

# # load the minst fashion dataset from kaggle
# if not os.path.exists('fashionmnist/fashion-mnist_test.csv'):
#     !rm -rf fashionmnist
#     !kaggle datasets download -d zalando-research/fashionmnist
#     !mkdir fashionmnist
#     !unzip fashionmnist.zip -d fashionmnist 

# Load the dataset
train_data = pd.read_csv('fashionmnist/fashion-mnist_train.csv')
test_data = pd.read_csv('fashionmnist/fashion-mnist_test.csv')

train_images = (train_data.drop('label', axis=1) / 255.0)
train_labels = train_data['label']

test_images = (test_data.drop('label', axis=1) / 255.0)
test_labels = test_data['label']

# train_images = train_data.iloc[:, 1:].values / 255.0
# train_labels = train_data.iloc[:, 0].values

# test_images = test_data.iloc[:, 1:].values / 255.0
# test_labels = test_data.iloc[:, 0].values

# Convert the dataframe to PyTorch tensors
# numpy.ndarray to torch.tensor that contains float values

train_images = torch.FloatTensor(train_images.values)
train_labels = torch.LongTensor(train_labels.values)
test_images = torch.FloatTensor(test_images.values)
test_labels = torch.LongTensor(test_labels.values)


# Create a neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() # Call the parent class's constructor
        self.fc1 = nn.Linear(28*28, 128) # 784 input features, 128 output features
        self.fc2 = nn.Linear(128, 128) # 128 input features, 128 output features
        self.fc3 = nn.Linear(128, 10) # 128 input features, 10 output features

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        x = F.softmax(self.fc2(x))
        x = self.fc3(x)
        return x

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.01)

net = Net()

for epoch in range(20):
    net.optimizer.zero_grad()
    output = net(train_images)
    net.loss = criterion(output, train_labels)
    net.loss.backward()
    writer.add_scalar("Loss/train", net.loss, epoch)
    writer.add_histogram("fc1.weight", net.fc1.weight, epoch)
    writer.add_histogram("fc2.weight", net.fc2.weight, epoch)
    net.optimizer.step()
    if epoch % 10 == 0:
        print('Epoch:', epoch, 'Loss:', net.loss.item())
    
y_pred = net(test_images)
_, predicted = torch.max(y_pred, 1)

print(accuracy_score(test_labels, predicted))
print(confusion_matrix(test_labels, predicted))

/tmp/ipykernel_755/3457683471.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc2(x))


Epoch: 0 Loss: 2.3036961555480957
Epoch: 10 Loss: 2.3036999702453613
0.1
[[   0    0    0    0    0    0 1000    0    0    0]
 [   0    0    0    0    0    0 1000    0    0    0]
 [   0    0    0    0    0    0 1000    0    0    0]
 [   0    0    0    0    0    0 1000    0    0    0]
 [   0    0    0    0    0    0 1000    0    0    0]
 [   0    0    0    0    0    0 1000    0    0    0]
 [   0    0    0    0    0    0 1000    0    0    0]
 [   0    0    0    0    0    0 1000    0    0    0]
 [   0    0    0    0    0    0 1000    0    0    0]
 [   0    0    0    0    0    0 1000    0    0    0]]
